# Labeling posts with Llama3-8b via Groq

We'll use the LiteLLM [Groq](https://litellm.vercel.app/docs/providers/groq) connection to connect to Groq and use that to label via Llama3-8b.


In [70]:
import json
import os
from pprint import pprint

import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

from ml_tooling.llm.inference import run_query

In [3]:
current_wd = os.getcwd()

### Let's load our data

In [4]:
with open("links_to_prompts_map_v2.json") as f:
    links_to_prompt_map = json.load(f)

In [5]:
links_prompts_lst: list[tuple[str, dict]] = [
    (link, prompt_dict)
    for (link, prompt_dict) in links_to_prompt_map.items()
]

Let's see what this looks like:

In [6]:
links_prompts_lst[0]

('https://bsky.app/profile/jbouie.bsky.social/post/3knqbtrdzrz2n',
 {'context_prompt': '\n\nPretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". \n\nThen, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are \'left-leaning\', \'moderate\', \'right-leaning\', or \'unclear\'. You are analyzing text that has been pre-identified as \'political\' in nature. If the text is not civic, return "unclear".\n\nT

#### Llama3-8b

Let's store all the. Llama3-8b results in one dictionary. Let's initialize that dictionary with the links and prompts from the input, and then for the cases where the prompt with context equals the prompt without context, we only run the query once in order to avoid duplication.

In [8]:
llama3_8b_results: dict = {}

In [9]:
for (link, prompt_dict) in links_to_prompt_map.items():
    llama3_8b_results[link] = prompt_dict
    llama3_8b_results[link]["context_llm_result"] = ""
    llama3_8b_results[link]["no_context_llm_result"] = ""

Let's create a mapping of links to prompts for both context and without context prompts, so we can just iterate through those during inference.

In [19]:
# we only want to run the context classification for posts
# that have context in the first place. If the prompts between
# the context and no-context cases are the 
link_to_context_prompt_map: dict = {
    link: prompt_dict["context_prompt"]
    for (link, prompt_dict) in llama3_8b_results.items()
    if not prompt_dict["prompts_are_equal"]
}

In [20]:
# spot-check and see if the prompt indeed has context.
# It appears to be correct. These should all have prompts
print(list(link_to_context_prompt_map.items())[0][0])
print(list(link_to_context_prompt_map.items())[0][1])

https://bsky.app/profile/jbouie.bsky.social/post/3knqbtrdzrz2n


Pretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". 

Then, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are 'left-leaning', 'moderate', 'right-leaning', or 'unclear'. You are analyzing text that has been pre-identified as 'political' in nature. If the text is not civic, return "unclear".

Think through your response step by step.


In [22]:
# we have 204 posts with context
len(link_to_context_prompt_map)

204

Now let's get the prompts without context

In [21]:
# we run the no-context prompts for all links
link_to_no_context_prompt_map: dict = {
    link: prompt_dict["no_context_prompt"]
    for (link, prompt_dict) in llama3_8b_results.items()
}

In [23]:
# spot-check and see if the prompt indeed has context.
# It appears to be correct.
print(list(link_to_context_prompt_map.items())[0][0])
print(list(link_to_context_prompt_map.items())[0][1])

https://bsky.app/profile/jbouie.bsky.social/post/3knqbtrdzrz2n


Pretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". 

Then, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are 'left-leaning', 'moderate', 'right-leaning', or 'unclear'. You are analyzing text that has been pre-identified as 'political' in nature. If the text is not civic, return "unclear".

Think through your response step by step.


In [24]:
# 354 without context.
len(link_to_no_context_prompt_map)

354

Let's now export these prompts

In [25]:
with open("link_to_prompts_with_context.json", 'w') as f:
    json.dump(link_to_context_prompt_map, f)

In [26]:
with open("link_to_prompts_with_no_context.json", 'w') as f:
    json.dump(link_to_no_context_prompt_map, f)

##### With context

Let's run inference for llama3-8b with context and export the results. Let's then compare it to the ground-truth labels.

In [27]:
llama3_8b_with_context_llm_results: list[str] = []

In [28]:
llama3_8b_with_context_llm_errors: list[dict] = []

In [ ]:
# run queries against Llama3-8b model. This will take a while since we're
# making a lot of requests to Groq. Takes ~10 minutes
# (but up to 30 minutes, depending on rate limits)
for idx, (link, prompt) in enumerate(link_to_context_prompt_map.items()):
    if idx % 50 == 0:
        print(f"Processing post {idx + 1} of {prompt}")
    try:
        result = run_query(prompt=prompt, model_name="Llama3-8b (via Groq)")
        llama3_8b_with_context_llm_results.append(result)
    except Exception as e:
        # for the ones that failed, this happens because we set a requirement
        # that it must be valid JSON. Some of the responses are not JSON, so
        # Groq throws an error and tells us that the result wasn't JSON.
        print(f"Error with post {link}: {e}")
        llama3_8b_with_context_llm_errors.append(
            {
                "idx": idx,
                "link": link,
                "prompt": prompt,
                "error": e
            }
        )
        llama3_8b_with_context_llm_results.append("")
        continue

Let's take a look at the results

In [30]:
len(llama3_8b_with_context_llm_results)

204

In [31]:
len(llama3_8b_with_context_llm_errors)

2

So, it couldn't classify 2 of the posts. These are the cases where it couldn't produce a valid JSON. We could easily just re-do these.

In [35]:
pprint(llama3_8b_with_context_llm_errors[0])

{'error': BadRequestError('GroqException - Error code: 400 - {\'error\': {\'message\': "Failed to generate JSON. Please adjust your prompt. See \'failed_generation\' for more details.", \'type\': \'invalid_request_error\', \'code\': \'json_validate_failed\', \'failed_generation\': \'{\\n    "civic": "civic",\\n    "political_ideology": "left-leaning",\\n    "reason_civic": "The post discusses a controversy surrounding the disposal of books featuring diverse characters and themes, highlighting issues of representation and censorship.",\\n    "reason_political_ideology": "The post\\\'s focus on the importance of representation and the potential for censorship aligns with left-leaning values."\'}}'),
 'idx': 109,
 'link': 'https://bsky.app/profile/clancyny.bsky.social/post/3knh2gspnxk2a',
 'prompt': '\n'
           '\n'
           'Pretend that you are a classifier that predicts whether a post has '
           'civic content or not. Civic refers to whether a given post is '
           're

Now, let's take the posts with context and their labels and export them. Let's only do the ones that have valid JSONs.

In [38]:
links_context_prompts = [
    (link, prompt)
    for (link, prompt) in link_to_context_prompt_map.items()
]

In [41]:
# let's verify that # of links and prompts = # of results
# n = 204 for both
print(len(links_context_prompts))
print(len(llama3_8b_with_context_llm_results))

204
204


Let's now record the links, prompts, and results

In [42]:
labels_with_links_and_context_prompts: dict[str, dict] = {
    link: {
        "context_prompt": prompt,
        "result": result
    }
    for ((link, prompt), result)
    in zip(
        links_context_prompts, llama3_8b_with_context_llm_results
    )
    if result != ""
}

Let's take a look at how it looks

In [43]:
example_link = links_context_prompts[0][0]
print(labels_with_links_and_context_prompts[example_link])

{'context_prompt': '\n\nPretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". \n\nThen, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are \'left-leaning\', \'moderate\', \'right-leaning\', or \'unclear\'. You are analyzing text that has been pre-identified as \'political\' in nature. If the text is not civic, return "unclear".\n\nThink through your response step by step.\n\nReturn in a JSON format 

Now let's dump these results.

In [45]:
with open("llama3_8b_context_prompts_results.json", 'w') as f:
    json.dump(labels_with_links_and_context_prompts, f)

Let's see how these look

In [46]:
with open("llama3_8b_context_prompts_results.json", 'r') as f:
    loaded_labels_with_links_and_context_prompts = json.load(f)

In [47]:
loaded_labels_with_links_and_context_prompts[example_link]

{'context_prompt': '\n\nPretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". \n\nThen, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are \'left-leaning\', \'moderate\', \'right-leaning\', or \'unclear\'. You are analyzing text that has been pre-identified as \'political\' in nature. If the text is not civic, return "unclear".\n\nThink through your response step by step.\n\nReturn in a JSON format 

Looks great!

### Let's process our results and then compare it to our ground truth data

First, let's load our ground-truth labels, remove NAs, and preprocess

In [48]:
ground_truth_labels: pd.DataFrame = pd.read_csv(
    "../manuscript_pilot/hand_labeled_pilot_posts.csv"
)

In [49]:
ground_truth_labels = ground_truth_labels[
    ~pd.isna(ground_truth_labels["civic_hand_label"])
]

In [50]:
ground_truth_labels_subset = ground_truth_labels[
    ["link", "civic_hand_label", "political_ideology_hand_label"]
]

Now let's process our labeled results and create a pandas dataframe. Let's transform our data into a list of dictionaries, and also process our JSON data as well.

In [58]:
llama3_8b_context_prompted_processed_labels: list[dict] = []

In [59]:
for (idx, (link, labeled_dict)) in enumerate(loaded_labels_with_links_and_context_prompts.items()):
    res = {}
    res["link"] = link
    res["prompt"] = labeled_dict["context_prompt"]
    res["json_result"] = labeled_dict["result"]
    result_dict = json.loads(labeled_dict["result"])
    res["valid_json_response"] = True
    res["hydrated_result"] = result_dict
    res["civic_label"] = result_dict["civic"]
    res["political_label"] = result_dict["political_ideology"]
    res["reason_civic_label"] = result_dict.get("reason_civic", None)
    res["reason_political_label"] = result_dict.get("reason_political_ideology", None)
    try:
        pass
    except Exception as e:
        print(f"Error with post {post['link']} at index {idx}: {e}")
        res["valid_json_response"] = False
        res["hydrated_result"] = None
        res["civic_label"] = None
        res["political_label"] = None
        res["reason_civic_label"] = None
        res["reason_political_label"] = None
    finally:
        llama3_8b_context_prompted_processed_labels.append(res)



Let's double-check that we have as many processed JSONs as we expect

In [60]:
print(len(llama3_8b_context_prompted_processed_labels))
print(len(llama3_8b_context_prompted_processed_labels) == len(loaded_labels_with_links_and_context_prompts))

202
True


Good! This looks correct. Let's export these now.

In [61]:
with open("llama3_8b_context_prompts_processed_results.json", 'w') as f:
    json.dump(llama3_8b_context_prompted_processed_labels, f)

Let's create a pandas dataframe out of these.

In [62]:
llama3_8b_context_prompts_df = pd.DataFrame(llama3_8b_context_prompted_processed_labels)

In [64]:
llama3_8b_context_prompts_df.head()

,link,prompt,json_result,valid_json_response,hydrated_result,civic_label,political_label,reason_civic_label,reason_political_label
0,https://bsky.app/profile/jbouie.bsky.social/po...,\n\nPretend that you are a classifier that pre...,"{\n ""civic"": ""civic"",\n ""political_ideol...",True,"{'civic': 'civic', 'political_ideology': 'left...",civic,left-leaning,The post references a specific policy proposal...,The post's language and tone are consistent wi...
1,https://bsky.app/profile/lethalityjane.bsky.so...,\n\nPretend that you are a classifier that pre...,"{\n ""civic"": ""civic"",\n ""political_ideol...",True,"{'civic': 'civic', 'political_ideology': 'uncl...",civic,unclear,None,None
2,https://bsky.app/profile/esqueer.bsky.social/p...,\n\nPretend that you are a classifier that pre...,"{\n ""civic"": ""civic"",\n ""political_ideol...",True,"{'civic': 'civic', 'political_ideology': 'left...",civic,left-leaning,"The post discusses censorship and neo-Nazism, ...",The post's language and tone suggest a left-le...
3,https://bsky.app/profile/bachynski.bsky.social...,\n\nPretend that you are a classifier that pre...,"{\n ""civic"": ""civic"",\n ""political_ideol...",True,"{'civic': 'civic', 'political_ideology': 'left...",civic,left-leaning,The post references a historical event (Great ...,The post's appeal to the conscience of Irish A...
4,https://bsky.app/profile/rainsurname.bsky.soci...,\n\nPretend that you are a classifier that pre...,"{\n ""civic"": ""civic"",\n ""political_ideol...",True,"{'civic': 'civic', 'political_ideology': 'left...",civic,left-leaning,None,None


In [65]:
llama3_8b_context_prompts_df.to_csv("llama3_8b_context_prompts_processed_results.csv")

### Great! Let's compare against the ground truth labels and get some metrics

Let's start by getting some basic metrics of our labels

In [66]:
print(llama3_8b_context_prompts_df["civic_label"].value_counts())

civic_label
civic        141
not civic     61
Name: count, dtype: int64


In [67]:
print(
    llama3_8b_context_prompts_df[
        llama3_8b_context_prompts_df["civic_label"] == "civic"
    ]["political_label"].value_counts()
)

political_label
left-leaning     86
unclear          35
right-leaning    15
moderate          5
Name: count, dtype: int64


Now let's compare these to our ground truth labels

In [68]:
llama3_8b_context_prompts_subset = (
    llama3_8b_context_prompts_df[["link", "civic_label", "political_label"]]
)

In [69]:
ground_truth_labels_subset.head()

,link,civic_hand_label,political_ideology_hand_label
0,https://bsky.app/profile/jbouie.bsky.social/po...,civic,left-leaning
1,https://bsky.app/profile/lethalityjane.bsky.so...,civic,unclear
2,https://bsky.app/profile/esqueer.bsky.social/p...,civic,left-leaning
3,https://bsky.app/profile/stuflemingnz.bsky.soc...,not civic,NaN
4,https://bsky.app/profile/sararoseg.bsky.social...,civic,left-leaning


In [73]:
joined_llama3_8b_context_results = pd.merge(
    llama3_8b_context_prompts_subset, ground_truth_labels_subset,
    on="link"
)

Let's export these results

In [ ]:
joined_llama3_8b_context_results.to_csv(
    "llama3_8b_context_results_joined_with_ground_truth.csv"
)

In [74]:
joined_llama3_8b_context_results.head()

,link,civic_label,political_label,civic_hand_label,political_ideology_hand_label
0,https://bsky.app/profile/jbouie.bsky.social/po...,civic,left-leaning,civic,left-leaning
1,https://bsky.app/profile/lethalityjane.bsky.so...,civic,unclear,civic,unclear
2,https://bsky.app/profile/esqueer.bsky.social/p...,civic,left-leaning,civic,left-leaning
3,https://bsky.app/profile/bachynski.bsky.social...,civic,left-leaning,civic,left-leaning
4,https://bsky.app/profile/rainsurname.bsky.soci...,civic,left-leaning,civic,left-leaning


In [75]:
joined_llama3_8b_context_results.shape

(202, 5)

In [76]:
llama3_8b_context_civic_metrics = precision_recall_fscore_support(
    y_true=joined_llama3_8b_context_results["civic_hand_label"].tolist(),
    y_pred=joined_llama3_8b_context_results["civic_label"].tolist(),
    average="binary",
    pos_label="civic"
)

In [77]:
(
    llama3_8b_context_civic_precision,
    llama3_8b_context_civic_recall,
    llama3_8b_context_civic_fbeta_score,
    llama3_8b_context_civic_support
) = llama3_8b_context_civic_metrics


In [79]:
llama3_8b_context_total_correct = sum([
    pred == true_label
    for (pred, true_label)
    in zip(
        joined_llama3_8b_context_results["civic_label"].tolist(),
        joined_llama3_8b_context_results["civic_hand_label"].tolist()
    )
])

In [80]:
llama3_8b_context_accuracy = (
    llama3_8b_context_total_correct / len(joined_llama3_8b_context_results)
)

In [81]:
print(f"Accuracy: {llama3_8b_context_accuracy}")

Accuracy: 0.8267326732673267


In [78]:
print(f"Precision: {llama3_8b_context_civic_precision}\tRecall: {llama3_8b_context_civic_recall}\tF-1 score: {llama3_8b_context_civic_fbeta_score}\tSupport: {llama3_8b_context_civic_support}")

Precision: 0.8297872340425532	Recall: 0.9140625	F-1 score: 0.8698884758364313	Support: None


Great! Now let's calculate for political ideology

### Let's run the code now for the non-context prompts.

Let's now run the same thing but for the non-context prompts.

We'll then have to do two analyses:
- Labeled performance of all non-context prompts vs. ground truth posts
- Labeled performance of posts that have context and non-context prompts, and see how they compare.

In [ ]:
loaded_labels_with_links_and_context_prompts[example_link]